In [8]:
pip install gspread pandas openai python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [45]:
# Import necessary libraries
import gspread
import pandas as pd
from dotenv import load_dotenv
import os
import json
import openai
from openai import ChatCompletion

# Load environment variables
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Authenticate with Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
client = gspread.service_account(filename="credentials.json")

# Load student profiles from Google Sheets
sheet = client.open("Youth Development").worksheet("Student Profiles")
student_profiles = pd.DataFrame(sheet.get_all_records())

# Display loaded profiles
print("Student Profiles Loaded Successfully!")
display(student_profiles)

# Function to find a student profile interactively
def get_student_profile():
    while True:
        student_name = input("\nEnter the name of the student (or type 'exit' to quit): ").strip().lower()
        if student_name == "exit":
            return None  # Exit the loop if the user types 'exit'

        # Check if the student name exists in the DataFrame
        matching_profiles = student_profiles[student_profiles["Name"].str.lower() == student_name]
        if not matching_profiles.empty:
            # Return the first matching profile as a dictionary
            return matching_profiles.iloc[0].to_dict()
        else:
            print("Student not found. Please try again or type 'exit' to quit.")

# Function to generate a personalized learning plan
def generate_learning_plan(student_profile):
    prompt = f"""
    Create a personalized weekly learning plan for a student with the following details:
    Name: {student_profile['Name']}
    Age: {student_profile['Age']}
    Grade: {student_profile['Grade']}
    Interests: {student_profile['Interests']}
    Challenges: {student_profile['Challenges']}
    
    Include:
    - Topics to cover each day.
    - Activities or assignments.
    - Time slots for each task.
    - Recommended resources (e.g., videos, books, or websites).
    """
    
    # Call OpenAI API
    response = ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an educational assistant AI."},
            {"role": "user", "content": prompt}
        ]
    )
    return response["choices"][0]["message"]["content"]

# Function to modify the learning plan based on feedback
def modify_plan(existing_plan, feedback):
    prompt = f"""
    Here is the current learning plan:
    {existing_plan}
    
    The user has requested the following changes: {feedback}.
    Update the plan accordingly while maintaining the learning objectives.
    """
    # Call OpenAI API to process feedback
    response = ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an educational assistant AI."},
            {"role": "user", "content": prompt}
        ]
    )
    return response["choices"][0]["message"]["content"]

# Adaptive Learning Plans Based on Progress
def adapt_learning_plan(student_profile):
    print("\nAdapting Learning Plan Based on Progress...")
    
    # Simulate progress tracking
    progress = {
        "Math": 70,  # Progress in percentage
        "Art": 50
    }
    student_profile["Progress"] = progress

    try:
        generate_prompt = f"""
        Create a personalized weekly learning plan for the student:
        {json.dumps(student_profile, indent=4)}
        Focus more on areas with lower progress percentages.
        """
        response = ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are an educational assistant AI."},
                {"role": "user", "content": generate_prompt}
            ]
        )
        adaptive_plan = response["choices"][0]["message"]["content"]
        print("\nAdaptive Learning Plan Generated Successfully:")
        print(adaptive_plan)
        return adaptive_plan
    except Exception as e:
        print(f"Error generating adaptive learning plan: {e}")
        return None

# Feedback Loop with Iterative Modification
def feedback_loop(learning_plan, student_name):
    print("\nStarting Feedback Loop for Iterative Modification...")
    while True:
        feedback = input("\nEnter your feedback to modify the plan (or type 'done' to finish): ").strip()
        if feedback.lower() == "done":
            print("\nFeedback Loop Complete!")
            break

        try:
            print("\nProcessing feedback...")
            updated_plan = modify_plan(learning_plan, feedback)
            print(f"\nUpdated Learning Plan for {student_name}:\n{updated_plan}")
            learning_plan = updated_plan  # Update for further iterations
        except Exception as e:
            print(f"Error during feedback processing: {e}")
            break

    print("\nFinal Learning Plan:")
    print(learning_plan)
    print("\nFeedback Loop ran successfully!")

# Main interactive workflow
def main():
    print("Welcome to the Learning Plan Generator!")

    while True:
        # Get the student profile interactively
        student_profile = get_student_profile()
        
        if student_profile is None:
            print("\nThank you for using the Learning Plan Generator. Goodbye!")
            break

        # Display the student profile
        print("\nStudent profile found:", student_profile)
        
        # Generate the learning plan
        print("\nGenerating learning plan...")
        learning_plan = generate_learning_plan(student_profile)
        print(f"\nLearning Plan for {student_profile['Name']}:\n")
        print(learning_plan)

        # **Adapt learning plan based on progress**
        adapted_plan = adapt_learning_plan(student_profile)
        if adapted_plan:
            learning_plan = adapted_plan  # Update learning plan if adapted

        # **Call the feedback loop**
        feedback_loop(learning_plan, student_profile['Name'])

# Run the interactive workflow
if __name__ == "__main__":
    main()


Student Profiles Loaded Successfully!


,Name,Age,Grade,Interests,Challenges
0,Alice,10,5th,"Math, Art",Struggles with fractions
1,Bob,12,7th,"Science, Robotics",Needs better time management
2,Charlie,9,4th,"Reading, Music",Shy during group activities
3,Diana,11,6th,"History, Sports",Difficulty focusing on tasks
4,Ethan,13,8th,"Programming, Chess",Overwhelmed with complex topics
5,Fiona,8,3rd,"Painting, Storytelling",Struggles with reading comprehension
6,George,10,5th,"Science, Astronomy",Lacks interest in classroom activities
7,Hannah,14,9th,"Creative Writing, Dance",Difficulty organizing ideas
8,Isaac,7,2nd,"Nature, Animals",Needs help with basic math skills
9,Jenny,12,7th,"Coding, Video Games",Struggles with group collaboration


Welcome to the Learning Plan Generator!

Student profile found: {'Name': 'Alice', 'Age': 10, 'Grade': '5th', 'Interests': 'Math, Art', 'Challenges': 'Struggles with fractions'}

Generating learning plan...

Learning Plan for Alice:

**Weekly Learning Plan for Alice**

**Monday:**

_Time slot:_ 10am - 11:30am (Math)
* _Topic:_ Introduction to fractions
* _Activity:_ Simple fraction worksheets (Equivalent fractions, Comparing fractions, Simplifying fractions)
* _Resource:_ Introduction to Fractions (Math is Fun website) & 'Fraction Action' book by Loreen Leedy

_Time slot:_ 1pm - 2:30pm (Art)
* _Topic:_ Basic sketching
* _Activity:_ Sketch an image from a picture book
* _Resource:_ 'You Can Draw in 30 Days' book by Mark Kistler.

**Tuesday:**

_Time slot:_ 10am - 11:30am (Math)
* _Topic:_ Basic arithmetic with fractions (Addition and Subtraction)
* _Activity:_ Solve exercises using fraction addition and subtraction
* _Resource:_ Khan Academy video tutorial about adding & subtracting frac